In [9]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.patches as patches
import matplotlib.colors as colors
import Q2Q1FEM
import PolyFEM
import pandas as pd 
import itertools
from copy import deepcopy
import amplify
from amplify import BinaryPoly, gen_symbols,sum_poly, decode_solution
from amplify import Solver
from amplify.client import FixstarsClient
token = "your token "
import time
import datetime
import os



In [2]:
def Plot_patch(nelm,fem_Node,fem_Element,p_plot,max_p,min_p,cmap):
    plt.figure()
    fem_NElem = fem_Element.shape[0]
    ax = plt.axes()
    cm = plt.get_cmap(cmap)
    for i in range(fem_NElem):
        c_ind = (p_plot[i]-min_p)/(max_p-min_p)
        r = patches.Rectangle(xy=fem_Node[int(fem_Element[i,0])], width=1/nelm, height=1/nelm, fc=colors.rgb2hex(cm(c_ind)),ec='w')
        ax.add_patch(r)
        plt.axis('scaled')
        ax.set_aspect('equal')
        
        

In [10]:
nelm = 32
wid = 1/3
Node,Element,NodeBC=Q2Q1FEM.RecSquareMeshQ2(nelm,wid)
Node_P,Element_P,NodeBC_P=Q2Q1FEM.RecSquareMeshQ1(nelm,wid) #Don't use NodeBC_P!!!!(NodeBC_P contains the value of the velocity🥶)
fem_NElem = Element.shape[0]
zIni = np.ones(fem_NElem)



In [11]:
adj_elem_list = []
for i in range(fem_NElem):
    t_adj_v = []
    t_adj_h = []
    if i%nelm!=0: ## up?? 
        t_u = i-1
        t_adj_v.append(t_u)
    if i>=nelm: ## left
        t_l = i-nelm
        t_adj_h.append(t_l)
    if i%nelm!=nelm-1: ## o?? below
        t_o = i+1
        t_adj_v.append(t_o)
    if (nelm*nelm)-i>nelm: ## right
        t_r = i+nelm
        t_adj_h.append(t_r)
    t_adj = [t_adj_v,t_adj_h]
    #print(t_adj)
    adj_elem_list.append(t_adj)
    
    

In [12]:
def Construct_QUBO(Nelm,u,v,volfrac,num_bit_per,el,ge,hev,dc):
    ## bit string
    v_limited = volfrac*Nelm

    ## coef for porous resistivity
    coef_pr = 10**4
    coef_pr_pres = [0 for _ in range(Nelm)]

    ## coef for energy loss, eqn constraint, gradient energy, heviside function, boundary constraint, domain constraints
    # alpha_el = 5
    # alpha_ge = 900.0
    # alpha_hev = 300.0
    alpha_el = el
    alpha_ge = ge
    alpha_hev = hev
    alpha_dc = dc

    ## qubit
    q = gen_symbols(BinaryPoly,Nelm,num_bit_per+1)

    ## objective function
    obj_func = BinaryPoly()

    ### energy loss
    for t_id in range(Nelm):
        coef_pr_e = (1-q[t_id][-1])*coef_pr
        t_uvec = u[t_id]
        t_vvec = v[t_id]
        obj_func += alpha_el*(coef_pr_e*(t_uvec*t_uvec+t_vvec*t_vvec) )
        
    ### gradient energy
    for t_id,t_q in enumerate(q):
        for tt_id in adj_elem_list[t_id]:
            for t_coord in range(2): # 2 if it's two-dimensional; in general, n if it's n-dimensional
                if len(tt_id) == 1:
                    phi_i = (sum_poly(t_q[:-1])/num_bit_per*2)-1
                    phi_j = (sum_poly(q[tt_id[0]][:-1])/num_bit_per*2)-1
                    t_obj = alpha_ge/2*((phi_i-phi_j)/2)**2
                elif len(tt_id) == 2:
                    phi_i = (sum_poly(q[tt_id[0]][:-1])/num_bit_per*2)-1
                    phi_j = (sum_poly(q[tt_id[1]][:-1])/num_bit_per*2)-1
                    t_obj = alpha_ge/2*(phi_i-phi_j)**2
                obj_func += t_obj

    ### heviside function
    for t_id, t_q in enumerate(q):
        obj_func += alpha_hev*(sum_poly(t_q[:-1])/num_bit_per - t_q[-1])**2

    ### domaint constraints
    obj_func += alpha_dc*(sum_poly([t[-1] for t in q]) - v_limited)**2
    
    client = FixstarsClient()
    client.parameters.timeout = 1000   # タイムアウト X m秒
    client.token = token # ローカル環境で使用する場合は、Amplify AEのアクセストークンを入力してください
    client.parameters.outputs.duplicate = True  # 同じエネルギー値の解を列挙するオプション（解が複数個あるため）

    solver = Solver(client)
    result = solver.solve(obj_func)

    solution = decode_solution(q, result[0].values)
    
    return solution



In [ ]:
result_dir = 'figdata_'+datetime.datetime.now().strftime('%Y%m%d%H%M%S')
os.mkdir(result_dir)

maxiter=10
num_bit_per = 9
volfrac = 0.5
dc = 160.
el = 1.

obj_list = []
frac_list = []
heav_list = []

for ge_num in range(10):
    hev = 5.*ge_num
    for hev_num in range(11):
        ge = hev_num*0.4
        start = time.time()
        p,f,u,v = Q2Q1FEM.Q2Q1FEM(Node,Element,NodeBC,Node_P,Element_P,NodeBC_P,zIni,q=0.01)
        z_QA_new = zIni
        obj = []
        frac = []
        heav = []
        for i in range(maxiter):
            z_QA = z_QA_new
            solution = Construct_QUBO(fem_NElem,u,v,volfrac,num_bit_per,el,ge,hev,dc)
            sol = []
            heviside=[]
            check=[]
            for t in solution:
                pred_t_d = np.sum(t[:-1])/num_bit_per
                sol.append(pred_t_d)
                pred_t_h = t[-1]
                heviside.append(pred_t_h)
                if pred_t_d >= 0.5:
                    check.append(1)
                else:
                    check.append(0) 
            heav.append((np.abs(np.array(check)-np.array(heviside))).sum())
            z_QA_new = np.array(heviside)
            z_QA_sol = np.array(sol)
            p,f_new,u,v = Q2Q1FEM.Q2Q1FEM(Node,Element,NodeBC,Node_P,Element_P,NodeBC_P,z_QA_new,q=1.)
            #print('f:{}'.format(f_new*1000))
            #print('Volume fraction: {}'.format(sum(heviside)/fem_NElem))
            obj.append(f_new*1000)
            frac.append(sum(heviside)/fem_NElem)
            if np.max(np.abs(z_QA-z_QA_new))<0.01:
                break
        end = time.time()
        #print('f:{}'.format(f_new*1000))
        np.save(result_dir+f'/array_el_{el}_ge_{ge}_hev_{hev}_dc_{dc}.npy',z_QA_new)
        #Plot_patch(nelm,Node_P,Element_P,z_QA_new,1,0,cmap='cool')
        plt.imshow(z_QA_new.reshape(32,32).T,cmap='cool')
        plt.savefig(result_dir+f'/el_{el}_ge_{ge}_hev_{hev}_dc_{dc}.png',format="png", dpi=200,bbox_inches='tight')
        #Plot_patch(nelm,Node_P,Element_P,z_QA_sol,1,0,cmap='cool')
        plt.imshow(z_QA_sol.reshape(32,32).T,cmap='cool')
        plt.savefig(result_dir+f'/sol_el_{el}_ge_{ge}_hev_{hev}_dc_{dc}.png',format="png", dpi=200,bbox_inches='tight')
        obj_list.append(obj)
        frac_list.append(frac)
        heav_list.append(heav)

np.save(result_dir+'/obj_list.npy',obj_list)
np.save(result_dir+'/frac_list.npy',frac_list)
np.save(result_dir+'/heav_list.npy',heav_list)

